In [159]:

# !pip install -qU langchain-openai

In [160]:
# ! pip install langchain_community

In [161]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough



In [162]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [163]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

def filter_messages(messages, k=10):
    return messages[-k:]
    
# with_message_history = RunnableWithMessageHistory(model, get_session_history)
# config = {"configurable": {"session_id": "4231"}}
# response = with_message_history.invoke(
#     [HumanMessage(content="Hi! I'm Bob")],
#     config=config,
# )
# response = with_message_history.invoke(
#     [HumanMessage(content="What's my name?")],
#     config=config,
# )

# response.content


In [164]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
config = {"configurable": {"session_id": "asdvsdv"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="안녕 내이름은 민우야")],"language":"Korean"},
    config=config,
)

response.content

'안녕하세요, 민우님! 무엇을 도와드릴까요?'

In [167]:
# response = with_message_history.invoke(
#     {"messages": [HumanMessage(content="What is my name?")],"language":"Korean"},
#     config=config,
# )

# response.content

'Your name is Minwoo.'

In [166]:
# response = with_message_history.invoke(
#     {"messages": [HumanMessage(content="내가 너에게 한 첫번째 말은?")],"language":"Korean"},
#     config=config,
# )

# response.content

'네, 민우님. 당신이 말씀하신 첫 번째 문장은 "안녕 내이름은 민우야" 였습니다. 부족한 저의 답변에도 불구하고 기억하고 있습니다. 어떻게 도와드릴까요?'